In [1]:
import soccerdata as sd
from joblib import Parallel, delayed
import pandas as pd
from tqdm import tqdm

[03/19/24 09:11:01] INFO     No custom team name replacements found. You can configure these in       ]8;id=880672;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=212281;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_config.py#87\87]8;;\
                             C:\Users\gfuen\soccerdata\config\teamname_replacements.json.                          

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=962272;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=611139;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_config.py#283\283]8;;\
                             C:\Users\gfuen\soccerdata\config\league_dict.json.                                    

In [3]:
print(sd.ESPN.available_leagues())

['ARG-PrimeraDivision', 'BRA-Brasileirao', 'CHI-PrimeraDivision', 'ENG-Premier League', 'ESP-La Liga', 'FRA-Ligue 1', 'GER-Bundesliga', 'ITA-Serie A', 'NED-Eredivisie', 'POR-PrimeiraDivisao', 'RUS-PremierLeague', 'SCO-Premiership', 'TUR-SuperLig', 'USA-MLS']


In [ ]:
# Arreglar : ---
# En serie
league = "ESP-La Liga"
seasons = [i for i in range(2000, 2023)]
espn = sd.ESPN(leagues=league, seasons=seasons, no_cache=True)
schedule = espn.read_schedule()
schedule

In [ ]:
ids_con_problemas = [14654,14653,15868,16446,17917,17924,53989,56397]
leagues = sd.ESPN.available_leagues()

for l in leagues[1:2]:
    seasons = [i for i in range(2000, 2023)]
    espn = sd.ESPN(leagues=l, seasons=seasons, no_cache=True)
    schedule = espn.read_schedule()
    while True:
        try:
            match_ids = list(set(schedule.game_id.unique())- set(ids_con_problemas))
            df = espn.read_matchsheet(match_ids)
            df.to_csv(f"ESPN_{l}.csv")
            print(f"Liga {l} lista")
            break
        except ConnectionError as err:
            print(f"La liga {l} presenta otro error")
            id_broken = str(err).split("=")[-1][:-1]
            ids_con_problemas.append(int(id_broken))
        except:
            print(f"La liga {l} presenta otro error")



In [14]:
df = pd.DataFrame()
lista_de_años = [i for i in range(2000, 2023)][::-1]
ids_con_problemas = [14654, 14653, 15868, 16446, 17917, 17924, 53989, 56397]
leagues = ["ESP-La Liga", 'ARG-PrimeraDivision', 'BRA-Brasileirao', 'CHI-PrimeraDivision',
           'POR-PrimeiraDivisao', 'RUS-PremierLeague', 'SCO-Premiership', 'TUR-SuperLig', 'USA-MLS']


def espn_scraper(league, lista_de_años):
    años_faltantes = []
    df = pd.DataFrame()
    ids_con_problemas = [14654, 14653, 15868, 16446, 17917, 17924, 53989, 56397]
    for a in lista_de_años:
        try:
            espn = sd.ESPN(leagues=league, seasons=a, no_cache=True)
            schedule = espn.read_schedule()
            match_ids = list(set(schedule.game_id.unique()) -
                             set(ids_con_problemas))
            data = espn.read_matchsheet(match_ids)
            df = pd.concat([df, data.reset_index()])
            print(f"La liga {league}, año {a} listo")
        except:
            print(f"La liga {league} presenta otro error, en el año {a}")
            años_faltantes.append(a)
    df.to_csv(f"ESPN_{league}.csv")
    return años_faltantes

In [16]:
n_workers = 9

lista_de_años = [i for i in range(2000, 2023)][::-1]
leagues = ["ESP-La Liga", 'ARG-PrimeraDivision', 'BRA-Brasileirao', 'CHI-PrimeraDivision',
           'POR-PrimeiraDivisao', 'RUS-PremierLeague', 'SCO-Premiership', 'TUR-SuperLig', 'USA-MLS']

resultados = Parallel(n_jobs=n_workers, verbose=10) (delayed(espn_scraper)(league, lista_de_años) for league in leagues)


[Parallel(n_jobs=9)]: Using backend LokyBackend with 9 concurrent workers.
[Parallel(n_jobs=9)]: Done   2 out of   9 | elapsed: 61.7min remaining: 215.9min
[Parallel(n_jobs=9)]: Done   3 out of   9 | elapsed: 63.7min remaining: 127.3min
[Parallel(n_jobs=9)]: Done   4 out of   9 | elapsed: 63.9min remaining: 79.9min
[Parallel(n_jobs=9)]: Done   5 out of   9 | elapsed: 72.6min remaining: 58.1min
[Parallel(n_jobs=9)]: Done   6 out of   9 | elapsed: 73.2min remaining: 36.6min
[Parallel(n_jobs=9)]: Done   7 out of   9 | elapsed: 84.2min remaining: 24.0min
[Parallel(n_jobs=9)]: Done   9 out of   9 | elapsed: 89.1min finished


In [19]:
años_faltantes = resultados
años_faltantes

[[2004, 2003, 2002],
 [2017, 2011, 2010, 2009, 2008, 2007, 2005, 2000],
 [2018, 2011, 2010, 2009, 2008, 2007],
 [2008, 2007, 2006],
 [2021, 2020, 2019, 2010, 2009, 2003, 2002, 2001, 2000],
 [2002, 2001, 2000],
 [2017, 2006, 2005, 2004, 2003, 2002, 2001, 2000],
 [2011, 2008, 2003, 2002, 2001, 2000],
 [2019, 2015, 2014, 2011, 2002, 2001, 2000]]

In [2]:
# Arreglar datos faltantes

ids_con_problemas = [14654, 14653, 15868, 16446, 17917, 17924, 53989, 56397]
leagues = ["ESP-La Liga", 'ARG-PrimeraDivision', 'BRA-Brasileirao', 'CHI-PrimeraDivision',
           'POR-PrimeiraDivisao', 'RUS-PremierLeague', 'SCO-Premiership', 'TUR-SuperLig', 'USA-MLS']
años_faltantes = [[2004, 2003, 2002],
                  [2017, 2011, 2010, 2009, 2008, 2007, 2005, 2000],
                  [2018, 2011, 2010, 2009, 2008, 2007],
                  [2008, 2007, 2006],
                  [2021, 2020, 2019, 2010, 2009, 2003, 2002, 2001, 2000],
                  [2002, 2001, 2000],
                  [2017, 2006, 2005, 2004, 2003, 2002, 2001, 2000],
                  [2011, 2008, 2003, 2002, 2001, 2000],
                  [2019, 2015, 2014, 2011, 2002, 2001, 2000]]


def solucionar_data_faltante(leagues, años_faltantes, i):
    ids_con_problemas = [14654, 14653, 15868, 16446, 17917, 17924, 53989, 56397]
    seasons = años_faltantes[i]
    espn = sd.ESPN(leagues=leagues[i], seasons=seasons, no_cache=True)
    try:
        schedule = espn.read_schedule()
    except:
        return
    match_ids = list(set(schedule.game_id.unique()) -
                    set(ids_con_problemas))
    data = pd.DataFrame()
    print(f"Total: {len(match_ids)}")
    for id in tqdm(match_ids):
        try:
            add = espn.read_matchsheet([id])
            data = pd.concat([data, add.reset_index()])
            print(len(data))
        except Exception as err:
            print(err)
            pass
    # Si pasa los match:
    df = pd.read_csv(f"ESPN_{leagues[i]}.csv")
    df = pd.concat([df, data.reset_index()])
    df.to_csv(f"ESPN_{leagues[i]}.csv")
    print(f"Liga {leagues[i]} lista")


In [4]:
n_workers = -1
data_faltante = Parallel(n_jobs=n_workers, verbose=10)(delayed(
    solucionar_data_faltante)(leagues, años_faltantes,i) for i in range(len(leagues)))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   9 | elapsed: 424.7min remaining: 1486.4min
